In [1]:
import re
import pandas as pd
import numpy as np
import calendar
import datetime
from datetime import date, datetime
import csv
def biggest(a, y, z):
    Max = a
    if y > Max:
        Max = y    
    if z > Max:
        Max = z
        if y > z:
            Max = y
    return Max

parse = lambda x: datetime.strptime(x,  '%m/%d/%Y')
df1=pd.read_csv("duplicate_with_locations_coordinate.csv")

df1 = df1.replace(np.nan, '', regex=True)
df1['charges'] = df1['charges'].str.replace(r"[|]+$", '')
df1['findings'] = df1['findings'].str.replace(r"[|]+$", '')
df1['sanctions/actions'] = df1['sanctions/actions'].str.replace(r"[|]+$", '')
condition = (df1['charges'].str.contains('|', na = False) | df1['findings'].str.contains('|', na = False))

rowwithpipe, rowwithoutpipe = df1[condition], df1[~condition]
rowwithoutpipe=rowwithoutpipe.iloc[:,1:]
rowwithpipe=rowwithpipe.iloc[:,1:]
dictionarylist= list(rowwithoutpipe.T.to_dict().values())


for index, row in rowwithpipe.iterrows():
    #print(row['charges'])
    charges=row['charges']
    findings=row['findings']
    sanctionsactions=row['sanctions/actions']
    role=row['role']
    listofcharges=list()
    listoffindings=list()
    listofsanctionsactions=[]
    if( (isinstance(charges, float)==False)):
        if(('|' in row["charges"])):
            listofcharges=row['charges'].split("|")
        elif(row['charges'].strip()!=""):
            listofcharges.append(row['charges'].strip())
            
    if( (isinstance(findings, float)==False)): 
        if('|' in row["findings"]):
            listoffindings=row['findings'].split("|")
        elif(row['findings'].strip()!=""):
            listoffindings.append(row['findings'].strip())
       
    if( (isinstance(sanctionsactions, float)==False) and ('|' in row["sanctions/actions"])):
        listofsanctionsactions=row['sanctions/actions'].split("|")
        
    numberofrow=biggest(len(listofcharges),len(listoffindings),len(listofsanctionsactions))
    
    listofnewrow=[]
    if(numberofrow>0):
        for i in range(0,numberofrow):
            series_copy = pd.Series(row.values.copy(), index=row.index.copy(),name=row.name)
            
            if(i<len(listofcharges)):
                series_copy['charges']=listofcharges[i]
            if(i<len(listoffindings)):
                series_copy['findings']=listoffindings[i]
            if(i<len(listofsanctionsactions)):
                series_copy['sanctions/actions']=listofsanctionsactions[i]
                
            if(len(listofcharges)<numberofrow and i>=len(listofcharges)):
                series_copy['charges']=""
            if(len(listoffindings)<numberofrow and i>=len(listoffindings)):
                series_copy['findings']=""
            if(len(listofsanctionsactions)<numberofrow and i>=len(listofsanctionsactions)):
                series_copy['sanctions/actions']=""
            
            newdict= dict(series_copy)
            dictionarylist.append(newdict)

            if(row['primarykey']== 'TA103'):
                print(listofcharges)
                print(listoffindings)
                print(i)
                print(numberofrow)
                print("============")
                print(newdict)
                        
    else:
        newdict= dict(row)
        dictionarylist.append(newdict)
        
with open('duplicate_with_locations_coordinate_breakdown.csv', 'w') as f:  # Just use 'w' mode in 3.x
    w = csv.DictWriter(f, dictionarylist[0].keys())
    w.writeheader()
    w.writerows(dictionarylist)

print("done")
#rowwithoutpipe.to_csv('Latest_HasBeenBreakdown.csv', index=False)

FileNotFoundError: File b'duplicate_with_locations_coordinate.csv' does not exist